In [ ]:
import os
__print__ = print
def print(string):
    os.system(f'echo \"{string}\"')
    __print__(string)

In [1]:
TEST_PATH = "../input/nta-landmarks-detection/baseline_NTA/data/test/" 
TRAIN_PATH = "../input/nta-landmarks-detection/baseline_NTA/data/train/"
SUBMISSION_PATH = "../input/nta-landmarks-detection/baseline_NTA/data/train/"
LANDMARKS = "../input/nta-landmarks-detection/baseline_NTA/data/train/landmarks.csv"

In [2]:
!pip install imutils

  Created wheel for imutils: filename=imutils-0.5.3-py3-none-any.whl size=25850 sha256=5d1b933a47fb3034745dbc356b7b440132aba1e5fcde0ff12a909ae6715df9a7
  Stored in directory: /root/.cache/pip/wheels/fc/9c/6d/1826267c72afa51b564c9c6e0f66abc806879338bc593a2270
Successfully built imutils


In [3]:
import dlib
print(dlib.__version__)

19.20.0


In [4]:
import os
import pandas as pd
import numpy as np
import cv2
import gc
import time
import random
import shutil 
from tqdm import tqdm
from torch.nn import functional as fnn
import torch
import torch.nn.functional as F
import math
import pickle
import glob
from PIL import Image
from skimage import io
import matplotlib.pyplot as plt
from pyparsing import *
from imutils import face_utils
import os.path
import csv
import re
from zipfile import ZipFile
from os.path import basename
import sys
import multiprocessing

In [5]:
landmarks_aug = pd.read_csv('../input/ntalandmarks/landmarks_aug_15_30_csv/landmarks_aug_15_30.csv')
landmarks_aug.drop('Unnamed: 0', axis = 1, inplace = True)
os.makedirs('./annotations_train_aug')

"landmarks_aug = pd.read_csv('../input/ntalandmarks/landmarks_aug_15_30_csv/landmarks_aug_15_30.csv')\nlandmarks_aug.drop('Unnamed: 0', axis = 1, inplace = True)\nos.makedirs('./annotations_train_aug')"

In [6]:
#Creating txt files with annotations from csv
#train aug annotations
for i in np.arange(landmarks_aug.shape[0]):
    print(i)
    s = re.sub('.jpg','',(str(landmarks_aug.iloc[i,0])+'\n'))
    print(s)
    for j in range(0,194*2,2):
        s+=(str(landmarks_aug.iloc[i,j+1])+' , '+str(landmarks_aug.iloc[i,j+2])+'\n')
    with open('/kaggle/working/annotations_train_aug/'+str(i+1)+".txt", "w") as file:
        file.write(s)
        
#zip all txt annots to zip
'''with ZipFile('annotations_train_aug_15_30.zip', 'w') as zipObj:
   # Iterate over all the files in directory
   for folderName, subfolders, filenames in os.walk('/kaggle/working/annotations_train_aug/'):
        for filename in filenames:
            #create complete filepath of file in directory
            filePath = os.path.join(folderName, filename)
            # Add file to zip
            zipObj.write(filePath, basename(filePath))
        zipObj.close()'''

"with ZipFile('annotations_train_aug_15_30.zip', 'w') as zipObj:\n   # Iterate over all the files in directory\n   for folderName, subfolders, filenames in os.walk('/kaggle/working/annotations_train_aug/'):\n        for filename in filenames:\n            #create complete filepath of file in directory\n            filePath = os.path.join(folderName, filename)\n            # Add file to zip\n            zipObj.write(filePath, basename(filePath))\n        zipObj.close()"

In [ ]:
import multiprocessing

options = dlib.shape_predictor_training_options()
options.cascade_depth = 16
options.tree_depth = 5
options.num_trees_per_cascade_level = 500
options.nu = 0.1
options.oversampling_amount = 20
options.feature_pool_size = 400
options.lambda_param = 0.1
options.num_test_splits = 20
options.feature_pool_region_padding = 0
#options.oversampling_translation_jitter=0.01
options.be_verbose = True
options.num_threads=multiprocessing.cpu_count()

In [ ]:
def center_crop(img, new_width=None, new_height=None):        

    width = img.shape[1]
    height = img.shape[0]

    if new_width is None:
        new_width = min(width, height)

    if new_height is None:
        new_height = min(width, height)

    left = int(np.ceil((width - new_width)/2))
    right = int(np.floor((width + new_width)/2))

    top = int(np.ceil((height - new_height)/2))
    bottom = int(np.floor((height + new_height)/2))

    return int(left),int(top),int(right),int(bottom)

In [ ]:
HELEN_IMGS_ABS_FILEPATH = '/kaggle/input/ntalandmarks/train_pngs_15_30/train_pngs/'
#HELEN_ANNOTATIONS_ABS_FILEPATH_TRAIN_ALL = '/kaggle/input/ntalandmarks/annotations_train_aug_15_30/'
HELEN_ANNOTATIONS_ABS_FILEPATH_TRAIN_ALL = '/kaggle/working/annotations_train_aug/'
face_cascade = cv2.CascadeClassifier('/kaggle/input/ntalandmarks/haarcascade/haarcascade_frontalface_default.xml')
helen_annotations_filelist_train_all = glob.glob(HELEN_ANNOTATIONS_ABS_FILEPATH_TRAIN_ALL+'*.txt')

xml_template_header = """<?xml version='1.0' encoding='ISO-8859-1'?>
<?xml-stylesheet type='text/xsl' href='image_metadata_stylesheet.xsl'?>
<dataset>
<name>train dataset</name>
<comment>Created by annakuchko</comment>
<images>\n"""

xml_template_footer = """</images>
</dataset>"""

def generate_xml(annots_path):
    xml = xml_template_header

    c = 0
    for file_name in annots_path:
        with open(file_name, 'r') as file:
            
            img_filename = HELEN_IMGS_ABS_FILEPATH + file.readline().replace('\n', '') + \
                '.png'  
            print(img_filename)

            image_xml = f"""  <image file='{img_filename}'>"""

            gray_image = cv2.imread(img_filename, cv2.IMREAD_GRAYSCALE)

            x, y, w, h = [int for i in range(4)]

            if len(face_position(gray_image)) == 0:
                print(
                    f"No faces detected: {img_filename}")
                x,y,w,h = center_crop(gray_image)
            else:
                x, y, w, h = face_position(gray_image)[0]
            w = x+w
            h = y+h

            image_xml += f"\n\t<box top = '{int(float(y))}' left = '{int(float(x))}' width = '{int(float(w))}' height = '{int(float(h))}'>\n"

            i = 0
            for line in file:
                x, y = line.replace('\n', '').replace(
                    '\r', '').replace(' ', '').split(',')
                image_xml += f"\t  <part name='{str(i).zfill(3)}' x='{int(float(x))}' y='{int(float(y))}'/>\n"
                i += 1
            image_xml += '\t</box>\n'
            image_xml += '  </image>\n'
            print(c)
            c+=1

            xml += image_xml

    xml += xml_template_footer
    return xml

def face_position(gray_img):
    faces = face_cascade.detectMultiScale(gray_img,
                                          scaleFactor=1.0005,
                                          minNeighbors=3,
                                          minSize=(
                                              int(len(gray_img[0]) / 2.5),
                                              int(len(gray_img) / 2.5))
                                         )
    return faces

In [ ]:
#generating xml
with open('annot_train_all_aug_15_30_xml.xml', 'w') as out_xml_file:
    xml = generate_xml(helen_annotations_filelist_train_all)
    out_xml_file.write(xml)

In [ ]:
#train_all_xml_filename = '/kaggle/input/ntalandmarks/annot_train_all_aug_15_30_xml.xml'
train_all_xml_filename = '/kaggle/working/annot_train_all_aug_15_30_xml.xml'

In [ ]:
#training dlib shape predictor ~2 hours
print("Start training")
dlib.train_shape_predictor(train_all_xml_filename, "train_all_15_30.dat", options)
print("Finish")

In [ ]:
def measure_model_error(model, xml_annotations):
    error = dlib.test_shape_predictor(xml_annotations, model)
    print("Error of the model: {} is {}".format(model, error))
    
measure_model_error('/kaggle/working/train_all_15_30.dat', train_all_xml_filename)

In [ ]:
lbls = pd.read_csv('/kaggle/input/nta-landmarks-detection/baseline_NTA/data/test/test_points.csv')

In [ ]:
sub = pd.DataFrame(np.zeros((330,194*2+1)))
predictor = dlib.shape_predictor("/kaggle/working/train_all_15_30.dat")
detector = dlib.get_frontal_face_detector()
ff_detector = dlib.get_frontal_face_detector()

sub_list = []
names_list = []
subzeroes = pd.read_csv('/kaggle/input/nta-landmarks-detection/SubmitAllZero.csv')
faces_folder = '/kaggle/input/ntalandmarks/test_pngs/test_pngs/'
print("Showing detections and predictions on the images in the faces folder...")


for i, f in enumerate(lbls['filename']):
    print(i)
    print("Processing file: {}".format(f))
    img = cv2.imread(faces_folder+f[:-4]+'.png', cv2.IMREAD_GRAYSCALE)
    dets = face_position(img)
    if len(dets)==0:
        ff_dets = ff_detector(img,1)
        if len(ff_dets)!=0:
            shape = predictor(img, ff_dets[0])
            shape = face_utils.shape_to_np(shape)
            print('Faces detected by ff detector')
        else:
            left, top, right, bottom = center_crop(img)
            dets = dlib.rectangle(int(left), int(top), int(left+right), int(top+bottom))
            shape = predictor(img, dets)
            shape = face_utils.shape_to_np(shape)
            print('No faces detected by cascade and frontal_face detector, use the whole img')

    else:
        x,y,w,h = dets[0]
        dets = dlib.rectangle(int(x), int(y), int(w+x), int(h+y))
        shape = predictor(img, dets)
        shape = face_utils.shape_to_np(shape)
        print('Faces detected by cascade detector')
    sub.iloc[i,0] = f[:-4]+'.jpg'
    sub.iloc[i,1:] = shape.ravel()
    sub_list.append(shape)
    names_list.append(f)

In [ ]:
landmarks = pd.read_csv('../input/nta-landmarks-detection/baseline_NTA/data/train/landmarks.csv')
sub.columns = landmarks.columns

df = pd.read_csv('../input/nta-landmarks-detection/baseline_NTA/data/test/test_points.csv')
for i in np.arange(330):
    c = [int(s) for s in df.iloc[i,1][1:-1].split(',')] 
    print(i)
    d = np.array(sub_list[i])
    subzeroes.iloc[i,1:] = d[c].ravel()
subzeroes.to_csv('submission.csv', index = False)